In [1]:
from sklearn.cluster import AgglomerativeClustering
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from scipy import spatial

In [2]:
from sklearn.neighbors.nearest_centroid import NearestCentroid

C:\Users\nadee\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.neighbors.nearest_centroid module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [33]:
data = pd.read_csv('FINAL_FINAL_SCALED_DATA.csv') 

df = pd.DataFrame(data, columns=['feature', 'feature_neutral', 'feature_frequency', 'feature_dissatisfaction','feature_satisfaction', 'hungerstation_e', 'h_dissatisfaction', 'h_satisfaction', 'jahez_e', 'j_dissatisfaction', 'j_satisfaction', 'shgardi_e', 's_dissatisfaction', 's_satisfaction', 'toyou_e', 't_dissatisfaction', 't_satisfaction', 'thechefz_e', 'c_dissatisfaction', 'c_satisfaction'])

In [4]:
for x in range(len(df)):
    if df['feature_satisfaction'][x] == 0 and df['feature_dissatisfaction'][x] == 0:
        print(df['feature'][x])
        df.drop(x, inplace=True)
        
df = df.reset_index(drop=True)

In [34]:
points = df[['feature_dissatisfaction','feature_satisfaction']].to_numpy()
# points = df[['feature_dissatisfaction','feature_satisfaction', 'hungerstation_e', 'h_dissatisfaction', 'h_satisfaction', 'jahez_e', 'j_dissatisfaction', 'j_satisfaction', 'shgardi_e', 's_dissatisfaction', 's_satisfaction', 'toyou_e', 't_dissatisfaction', 't_satisfaction', 'thechefz_e', 'c_dissatisfaction', 'c_satisfaction']].to_numpy()

# 1. Experiments

## 3 clusters

In [6]:
cluster = AgglomerativeClustering(n_clusters=3, affinity='euclidean', linkage='ward')
y = cluster.fit_predict(points)
df['cluster'] = cluster.labels_

In [7]:
table = {'feature': [], "cluster": [], "feature_dissatisfaction": [], "feature_satisfaction": [], "feature_neutral": [], "feature_frequency": [], 'hungerstation_e': [], 'h_dissatisfaction':[], 'h_satisfaction': [], 'jahez_e': [], 'j_dissatisfaction':[], 'j_satisfaction': [], 'shgardi_e': [], 's_dissatisfaction':[], 's_satisfaction': [], 'toyou_e': [], 't_dissatisfaction':[], 't_satisfaction': [], 'thechefz_e': [], 'c_dissatisfaction':[], 'c_satisfaction': []}

for x in range(len(df)):
        table["feature"].append(df['feature'][x])
        table["cluster"].append(df['cluster'][x])
        
        table["feature_dissatisfaction"].append(df['feature_dissatisfaction'][x])
        table["feature_satisfaction"].append(df['feature_satisfaction'][x])
        table["feature_neutral"].append(df['feature_neutral'][x])
        table["feature_frequency"].append(df['feature_frequency'][x])
            
        table["hungerstation_e"].append(df['hungerstation_e'][x])
        table["h_dissatisfaction"].append(df['h_dissatisfaction'][x])
        table["h_satisfaction"].append(df['h_satisfaction'][x])

        table["jahez_e"].append(df['jahez_e'][x])
        table["j_dissatisfaction"].append(df['j_dissatisfaction'][x])
        table["j_satisfaction"].append(df['j_satisfaction'][x])

        table["shgardi_e"].append(df['shgardi_e'][x])
        table["s_dissatisfaction"].append(df['s_dissatisfaction'][x])
        table["s_satisfaction"].append(df['s_satisfaction'][x])

        table["toyou_e"].append(df['toyou_e'][x])
        table["t_dissatisfaction"].append(df['t_dissatisfaction'][x])
        table["t_satisfaction"].append(df['t_satisfaction'][x])

        table["thechefz_e"].append(df['thechefz_e'][x])
        table["c_dissatisfaction"].append(df['c_dissatisfaction'][x])
        table["c_satisfaction"].append(df['c_satisfaction'][x])

In [8]:
df=pd.DataFrame.from_dict(table,orient='index').transpose()
df.to_csv('ah_3_clusters_2v_NEW.csv', index = False)

## 4 clusters 

In [23]:
cluster = AgglomerativeClustering(n_clusters=4, affinity='euclidean', linkage='ward')
y = cluster.fit_predict(points)
df['cluster'] = cluster.labels_

In [24]:
table = {'feature': [], "cluster": [], "feature_dissatisfaction": [], "feature_satisfaction": [], "feature_neutral": [], "feature_frequency": [], 'hungerstation_e': [], 'h_dissatisfaction':[], 'h_satisfaction': [], 'jahez_e': [], 'j_dissatisfaction':[], 'j_satisfaction': [], 'shgardi_e': [], 's_dissatisfaction':[], 's_satisfaction': [], 'toyou_e': [], 't_dissatisfaction':[], 't_satisfaction': [], 'thechefz_e': [], 'c_dissatisfaction':[], 'c_satisfaction': []}

for x in range(len(df)):
        table["feature"].append(df['feature'][x])
        table["cluster"].append(df['cluster'][x])
        
        table["feature_dissatisfaction"].append(df['feature_dissatisfaction'][x])
        table["feature_satisfaction"].append(df['feature_satisfaction'][x])
        table["feature_neutral"].append(df['feature_neutral'][x])
        table["feature_frequency"].append(df['feature_frequency'][x])
            
        table["hungerstation_e"].append(df['hungerstation_e'][x])
        table["h_dissatisfaction"].append(df['h_dissatisfaction'][x])
        table["h_satisfaction"].append(df['h_satisfaction'][x])

        table["jahez_e"].append(df['jahez_e'][x])
        table["j_dissatisfaction"].append(df['j_dissatisfaction'][x])
        table["j_satisfaction"].append(df['j_satisfaction'][x])

        table["shgardi_e"].append(df['shgardi_e'][x])
        table["s_dissatisfaction"].append(df['s_dissatisfaction'][x])
        table["s_satisfaction"].append(df['s_satisfaction'][x])

        table["toyou_e"].append(df['toyou_e'][x])
        table["t_dissatisfaction"].append(df['t_dissatisfaction'][x])
        table["t_satisfaction"].append(df['t_satisfaction'][x])

        table["thechefz_e"].append(df['thechefz_e'][x])
        table["c_dissatisfaction"].append(df['c_dissatisfaction'][x])
        table["c_satisfaction"].append(df['c_satisfaction'][x])

In [25]:
df=pd.DataFrame.from_dict(table,orient='index').transpose()
df.to_csv('ah_4_clusters_2v_NEW.csv', index = False)

## 5 clusters 

In [35]:
cluster = AgglomerativeClustering(n_clusters=5, affinity='euclidean', linkage='ward')
y = cluster.fit_predict(points)
df['cluster'] = cluster.labels_

In [36]:
table = {'feature': [], "cluster": [], "feature_dissatisfaction": [], "feature_satisfaction": [], "feature_neutral": [], "feature_frequency": [], 'hungerstation_e': [], 'h_dissatisfaction':[], 'h_satisfaction': [], 'jahez_e': [], 'j_dissatisfaction':[], 'j_satisfaction': [], 'shgardi_e': [], 's_dissatisfaction':[], 's_satisfaction': [], 'toyou_e': [], 't_dissatisfaction':[], 't_satisfaction': [], 'thechefz_e': [], 'c_dissatisfaction':[], 'c_satisfaction': []}

for x in range(len(df)):
        table["feature"].append(df['feature'][x])
        table["cluster"].append(df['cluster'][x])
        
        table["feature_dissatisfaction"].append(df['feature_dissatisfaction'][x])
        table["feature_satisfaction"].append(df['feature_satisfaction'][x])
        table["feature_neutral"].append(df['feature_neutral'][x])
        table["feature_frequency"].append(df['feature_frequency'][x])
            
        table["hungerstation_e"].append(df['hungerstation_e'][x])
        table["h_dissatisfaction"].append(df['h_dissatisfaction'][x])
        table["h_satisfaction"].append(df['h_satisfaction'][x])

        table["jahez_e"].append(df['jahez_e'][x])
        table["j_dissatisfaction"].append(df['j_dissatisfaction'][x])
        table["j_satisfaction"].append(df['j_satisfaction'][x])

        table["shgardi_e"].append(df['shgardi_e'][x])
        table["s_dissatisfaction"].append(df['s_dissatisfaction'][x])
        table["s_satisfaction"].append(df['s_satisfaction'][x])

        table["toyou_e"].append(df['toyou_e'][x])
        table["t_dissatisfaction"].append(df['t_dissatisfaction'][x])
        table["t_satisfaction"].append(df['t_satisfaction'][x])

        table["thechefz_e"].append(df['thechefz_e'][x])
        table["c_dissatisfaction"].append(df['c_dissatisfaction'][x])
        table["c_satisfaction"].append(df['c_satisfaction'][x])

In [37]:
df=pd.DataFrame.from_dict(table,orient='index').transpose()
df.to_csv('ah_5_clusters_2v_NEW.csv', index = False)

# 2. Evaluation

In [38]:
clf = NearestCentroid()
clf.fit(points, y)
print("Centroids:")
centroids = clf.centroids_
print(centroids)

Centroids:
[[0.0197189  0.61995877]
 [0.01028791 0.39114392]
 [0.02367078 0.13746826]
 [1.         1.        ]
 [0.05454649 0.00530295]]


In [39]:
labels = cluster.labels_

In [40]:
cluster_sizes = np.bincount(labels)

In [41]:
cluster_sizes

array([ 4,  5, 10,  1,  8], dtype=int64)

## 2.1 Dunn index 

In [13]:
import pylab as pl
from scipy.spatial import ConvexHull

In [14]:
# compute the diameter based on convex hull 
def diameter(pts):
  # need at least 3 points to construct the convex hull
  if pts.shape[0] <= 1:
    return 0
  if pts.shape[0] == 2:
    return ((pts[0] - pts[1])**2).sum()
  # two points which are fruthest apart will occur as vertices of the convex hull
  hull = ConvexHull(pts)
  candidates = pts[ConvexHull(pts).vertices]
  
  return spatial.distance_matrix(candidates, candidates).max()

def dunn_index(pts, labels, centroids):
  max_intracluster_dist = max(diameter(pts[labels==i]) for i in np.unique(labels))
  # get pairwise distances between centroids
  cluster_dmat = spatial.distance_matrix(centroids, centroids)
  # fill diagonal with +inf: ignore zero distance to self in "min" computation
  np.fill_diagonal(cluster_dmat, np.inf)
  min_intercluster_dist = cluster_sizes.min()
  return min_intercluster_dist / max_intracluster_dist

In [42]:
dunn_index(points, labels, centroids)

4.387661277940202

## 2.2 Silhouette index 

In [16]:
from sklearn.metrics import silhouette_score

In [43]:
s_score = silhouette_score(points, labels)
print("Silhouette score =", s_score)

Silhouette score = 0.5210780321298146


## 2.3 Calinski Harabasz  

In [18]:
from sklearn.metrics import calinski_harabasz_score

In [44]:
ch_score = calinski_harabasz_score(points, labels)
print("Calinski Harabasz score =", ch_score)

Calinski Harabasz score = 200.444882000808
